# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## Settings

In [4]:
weight_settings = [None, 1., 10., 100.]
weight_setting_filenames = ['', '_w1', '_w10', '_w100']

## FisherInformation instances

In [5]:
fisher_all = FisherInformation(sample_dir + 'samples.h5', debug=True)
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5', debug=True)

11:55  
11:55  ------------------------------------------------------------
11:55  |                                                          |
11:55  |  MadMiner v2018.11.02                                    |
11:55  |                                                          |
11:55  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
11:55  |                                                          |
11:55  ------------------------------------------------------------
11:55  
11:55  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
11:55  Found 2 parameters:
11:55     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.1, 0.1))
11:55     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.1, 0.1))
11:55  Found 6 benchmarks:
11:55     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
11:55     morphing_basis_vector_1: cWWW = -9.39e-02, cWWWtilde = 0.07
11:55     morphing_basis_vector_2: cWWW = 0.

## Truth-level info

In [6]:
fisher_info, cov = fisher_all.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton.npy', cov)


In [7]:
fisher_info, cov = fisher_tight.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton_tight.npy', cov)


## Rate-only info

In [8]:
fisher_info, cov = fisher_all.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec.npy', cov)


In [9]:
fisher_info, cov = fisher_tight.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec_tight.npy', cov)



## Full Fisher info (ML)

In [10]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full{}.npy'.format(weight_filename), cov)

12:00  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/ensemble.json
12:00  Found ensemble with 10 estimators and expectations [[ 0.07598135  0.05132959]
 [ 0.00661409 -0.01126869]
 [ 0.01919232  0.0195618 ]
 [-0.00168112  0.02560593]
 [ 0.01931735  0.00392584]
 [ 0.00483713  0.01156073]
 [ 0.0191037  -0.02803848]
 [ 0.04548576 -0.0297866 ]
 [ 0.00432217 -0.02863304]
 [ 0.00968069  0.01947241]]
12:00  Evaluating Fisher information for 10 estimators in ensemble
12:00  Starting evaluation for estimator 1 / 10 in ensemble
12:00  Expected score (should be close to zero): [0.07482558 0.05189442]
12:00  Starting evaluation for estimator 2 / 10 in ensemble
12:00  Expected score (should be close to zero): [ 0.00288686 -0.0058842 ]
12:00  Starting evaluation for estimator 3 / 10 in ensemble
12:01  Expected score (should be close to zero): [0.02044738 0.01895345]
12:01  Starting evaluation for estimator 4 / 10 in ensemb

In [11]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full_tight{}.npy'.format(weight_filename), cov)
    

12:03  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all_tight/ensemble.json
12:03  Found ensemble with 10 estimators and expectations [[-0.21800934  0.16104713]
 [ 0.26496118  0.25469989]
 [-0.2558946   0.45348474]
 [-0.00911883  0.26716518]
 [ 0.10316634  0.12745135]
 [ 0.07300109  0.22365138]
 [-0.08684525 -0.01064833]
 [ 0.21209612  0.10127637]
 [ 0.08032635  0.23434745]
 [-0.05892296  0.32795236]]
12:03  Evaluating Fisher information for 10 estimators in ensemble
12:03  Starting evaluation for estimator 1 / 10 in ensemble
12:03  Expected score (should be close to zero): [-0.16422392  0.01718709]
12:03  Starting evaluation for estimator 2 / 10 in ensemble
12:04  Expected score (should be close to zero): [0.3455682  0.06602671]
12:04  Starting evaluation for estimator 3 / 10 in ensemble
12:04  Expected score (should be close to zero): [-0.30928487  0.326418  ]
12:04  Starting evaluation for estimator 4 / 10 i

In [12]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight_sgd',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full_tight_sgd{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full_tight_sgd{}.npy'.format(weight_filename), cov)
    

14:47  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all_tight_sgd/ensemble.json
14:47  Found ensemble with 10 estimators and expectations [[-0.0925543  -0.8873778 ]
 [-0.72393084  0.24227424]
 [-0.10777807 -0.0946581 ]
 [-0.73415375  0.40563491]
 [-0.13606827 -1.11668205]
 [ 1.07897842  0.12356748]
 [-0.20122971 -0.30029571]
 [ 1.03816295  0.50730717]
 [-0.47743225  0.68404365]
 [-0.66639858  0.74667388]]
14:47  Evaluating Fisher information for 10 estimators in ensemble
14:47  Starting evaluation for estimator 1 / 10 in ensemble
14:47  Expected score (should be close to zero): [-0.12765652 -0.891179  ]
14:47  Starting evaluation for estimator 2 / 10 in ensemble
14:48  Expected score (should be close to zero): [-0.7443589   0.25837833]
14:48  Starting evaluation for estimator 3 / 10 in ensemble
14:48  Expected score (should be close to zero): [-0.10953198 -0.25226912]
14:48  Starting evaluation for estimator 4 

## Info in observables (ML)

means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_resurrection_tight',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

print(means[0])

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_resurrection_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_resurrection_tight{}.npy'.format(weight_filename), cov)
    

## Histogram info

In [13]:
filenames = ['met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la', 'phi_resurrection']
observables = ['et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_met', 'deltaphi_l1_a1', 'phi_resurrection']
bins = 25

In [14]:
for filename, obs in zip(filenames, observables):
    info, cov = fisher_tight.calculate_fisher_information_hist1d(
        theta=[0.,0.],
        luminosity=300000.,
        observable=obs,
        nbins=bins,
        histrange=None
    )
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    np.save(result_dir + 'information_covariance_histo_{}_tight.npy'.format(filename), cov)
    

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:883: RuntimeWarning: divide by zero encountered in true_divide
  inv_sigma = sanitize_array(1.0 / sigma)  # Shape (n_events,)
